In [1]:
## 1.Imports & Global Setup
import tkinter as tk
from tkinter import Toplevel, Label, Menu, Button, PhotoImage
from PIL import Image, ImageTk
import cv2
from deepface import DeepFace
import os
import numpy as np
import imageio
import glob
import threading
import datetime

# === Emoji & Save Paths ===
emoji_dir = r"C:\Users\moham\Downloads\Emotion_Recognition_Project\Real-time-Facial-Emotion-Recognition-using-OpenCV-and-Deepface\Emoji"
save_path = r"C:\Users\moham\Downloads\Emotion_Captures"
image_path = r"C:\Users\moham\Downloads\Emotion_Recognition_Project\Real-time-Facial-Emotion-Recognition-using-OpenCV-and-Deepface\image\face_emotion.png"
os.makedirs(save_path, exist_ok=True)


In [2]:
## 2.Load Emojis, Fonts, GUI Setup
# === Load emoji gifs and icons ===
emoji_gifs = {}
emoji_icons = {}
for path in glob.glob(os.path.join(emoji_dir, "*.gif")):
    emotion = os.path.splitext(os.path.basename(path))[0].lower()
    emoji_gifs[emotion] = [cv2.cvtColor(frame, cv2.COLOR_RGBA2BGR) for frame in imageio.mimread(path)]

for path in glob.glob(os.path.join(emoji_dir, "*.png")):
    emotion = os.path.splitext(os.path.basename(path))[0].lower()
    emoji_icons[emotion] = cv2.resize(cv2.imread(path), (24, 24))

font = cv2.FONT_HERSHEY_SIMPLEX
video_writer = None
recording = False

# === GUI SETUP ===
root = tk.Tk()
root.title("Real-Time Face Emotion Recognition")
root.geometry("900x500")


''

In [3]:
## Cell 3: GUI Layout & Menu
# === LEFT PANEL ===
left_frame = tk.Frame(root, width=400, bg="white")
left_frame.pack(side="left", fill="both")

Label(left_frame, text="Real-Time Face Emotion Recognition", font=("Helvetica", 14, "bold"), bg="white").pack(pady=30)

camera_icon = PhotoImage(file="camera_icon.png") if os.path.exists("camera_icon.png") else None
capture_icon = PhotoImage(file="capture_icon.png") if os.path.exists("capture_icon.png") else None
record_icon = PhotoImage(file="record_icon.png") if os.path.exists("record_icon.png") else None
exit_icon = PhotoImage(file="exit_icon.png") if os.path.exists("exit_icon.png") else None

Button(left_frame, text="😀 Start Prediction", font=("Helvetica", 12), bg="#5cb85c", fg="white", command=lambda: threading.Thread(target=detect_emotions).start()).pack(pady=10)
Button(left_frame, text="❌ Exit", font=("Helvetica", 12), bg="#0275d8", fg="white", command=root.quit).pack(pady=10)

# === RIGHT PANEL ===
right_frame = tk.Frame(root)
right_frame.pack(side="right", fill="both", expand=True)
try:
    img = Image.open(image_path)
    img = img.resize((480, 480))
    img_tk = ImageTk.PhotoImage(img)
    Label(right_frame, image=img_tk).pack()
except:
    Label(right_frame, text="Image could not be loaded", font=("Helvetica", 14)).pack(pady=200)

# === MENUBAR ===
menu_bar = Menu(root)
root.config(menu=menu_bar)
menu_bar.add_command(label="User Manual", command=lambda: show_manual())
menu_bar.add_command(label="App Benefits", command=lambda: show_benefits())


In [4]:
## Cell 4: Helper Functions (manuals, detection, etc.)
def show_manual():
    manual_win = Toplevel(root)
    manual_win.title("User Manual")
    manual_win.geometry("420x330")
    Button(manual_win, text="⬅ Back", bg="#f0ad4e", command=manual_win.destroy).place(x=10, y=5)
    Label(manual_win, text="📖 User Manual", font=("Helvetica", 13, "bold")).pack(pady=10)
    Label(manual_win, text=(
        "📷 This app uses your webcam to detect facial emotions in real-time\n"
        "▶ Press 'q' to exit the emotion detection window\n"
        "💾 Press 'w' to capture and save the current frame\n"
        "🔍 Emotions are displayed on the screen using DeepFace analysis\n"
        "🧠 Uses advanced deep learning for accurate emotion prediction"
    ), font=("Helvetica", 11), justify="left").pack(padx=15, pady=10)

def show_benefits():
    benefits_win = Toplevel(root)
    benefits_win.title("App Benefits")
    benefits_win.geometry("420x300")
    Button(benefits_win, text="⬅ Back", bg="#5bc0de", command=benefits_win.destroy).place(x=10, y=5)
    Label(benefits_win, text="✨ App Benefits", font=("Helvetica", 13, "bold")).pack(pady=10)
    Label(benefits_win, text=(
        "😊 Real-time facial emotion detection\n"
        "🖥️ Simple and user-friendly interface\n"
        "🤖 Powered by deep learning and AI (DeepFace)\n"
        "📊 Displays dominant emotion with overlay\n"
        "🖼️ Option to capture snapshots during detection\n"
        "🎯 Useful for emotion-aware applications and analysis\n"
        "💡 Great for research and educational purposes"
    ), font=("Helvetica", 11), justify="left").pack(padx=15, pady=10)

def toggle_video_record():
    global recording
    recording = not recording

def capture_image():
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    if ret:
        file_name = f"capture_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.jpg"
        path = os.path.join(save_path, file_name)
        cv2.imwrite(path, frame)
        cv2.imshow("Captured Image", frame)
        cv2.waitKey(1500)
        cv2.destroyAllWindows()
    cap.release()

def detect_emotions():
    global video_writer, recording
    cap = cv2.VideoCapture(0)
    emoji_frame_index = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, 1.1, 5)
        face_count = len(faces)

        dominant_emotion = "No Face"
        emotion_scores = None

        for (x, y, w, h) in faces:
            face_roi = rgb[y:y + h, x:x + w]

            try:
                result = DeepFace.analyze(face_roi, actions=['emotion'], enforce_detection=False)
                dominant_emotion = result[0]['dominant_emotion'].lower()
                emotion_scores = result[0]['emotion']

                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, dominant_emotion.upper(), (x, y - 10), font, 0.7, (0, 255, 0), 2)
            except Exception as e:
                print("[WARNING] Emotion detection error:", e)
            break

        panel_width = 240
        panel = np.ones((frame.shape[0], panel_width, 3), dtype=np.uint8) * 255

        if dominant_emotion in emoji_gifs:
            emoji_frames = emoji_gifs[dominant_emotion]
            emoji_img = emoji_frames[emoji_frame_index % len(emoji_frames)]
            emoji_frame_index += 1
            emoji_resized = cv2.resize(emoji_img, (144, 144))
            panel[10:10 + 144, 48:48 + 144] = emoji_resized
            cv2.putText(panel, dominant_emotion.upper(), (65, 170), font, 0.6, (0, 0, 255), 2)

        if emotion_scores:
            sorted_emotions = sorted(emotion_scores.items(), key=lambda item: -item[1])
            bar_start_y = 200
            bar_height = 18
            spacing = 6
            max_bar_width = 100

            for idx, (emo, val) in enumerate(sorted_emotions):
                y = bar_start_y + idx * (bar_height + spacing)
                if emo in emoji_icons:
                    panel[y:y + 24, 10:34] = emoji_icons[emo]
                bar_len = int((val / 100) * max_bar_width)
                cv2.rectangle(panel, (40, y + 5), (140, y + 5 + bar_height), (230, 230, 230), -1)
                cv2.rectangle(panel, (40, y + 5), (40 + bar_len, y + 5 + bar_height), (0, 100, 255), -1)
                cv2.putText(panel, f"{emo.upper()} : {int(val)}%", (145, y + 20), font, 0.4, (0, 0, 0), 1)

        if recording:
            if video_writer is None:
                fourcc = cv2.VideoWriter_fourcc(*'XVID')
                filename = f"video_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.avi"
                filepath = os.path.join(save_path, filename)
                video_writer = cv2.VideoWriter(filepath, fourcc, 10, (frame.shape[1], frame.shape[0]))
            video_writer.write(frame)

        cv2.putText(frame, f"Faces: {face_count}", (10, 30), font, 0.7, (255, 255, 255), 2)
        combined = np.hstack((panel, frame))
        cv2.imshow("Real-Time Emotion Detection", combined)

        key = cv2.waitKey(100) & 0xFF
        if key == ord('q'):
            break

    if video_writer:
        video_writer.release()
        video_writer = None
    recording = False
    cap.release()
    cv2.destroyAllWindows()


In [6]:
 ##Cell 5: Run App (Canvas Launch)
# === Launch the GUI (canvas window opens here) ===
root.mainloop()
